In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import textwrap

from langchain.llms import OpenAI
MODEL = "text-davinci-003"
llm = OpenAI(temperature=0, model_name=MODEL)

In [6]:
# PromptTemplate
from langchain import PromptTemplate
simple_answer_prompt = PromptTemplate.from_template("""
Answer the question below. Some information is provided. 

Question: {question}

Relevant Information: {relevant_information}
""")

final_prompt = simple_answer_prompt.format(question="What is the capital of France?", relevant_information="France is a country in Europe.")

print(final_prompt)
print("--------------------------------------------------")
print(llm(final_prompt))


Answer the question below. Some information is provided. 

Question: What is the capital of France?

Relevant Information: France is a country in Europe.

--------------------------------------------------

Answer: Paris is the capital of France.


In [13]:
# chat
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

MODEL = "gpt-3.5-turbo"

chat = ChatOpenAI(temperature=0, model_name=MODEL)
history = ChatMessageHistory()

QUESTION = "Who holds the record in 400-meter individual medley?"
print(f"Question: {textwrap.fill(QUESTION, 50)}")
history.add_user_message(QUESTION)

response = chat(history.messages)

print(f"Answer: {textwrap.fill(response.content, 50)}")
history.add_ai_message(response.content)

print('\n\n')
QUESTION = "But I recently read Leon Marchand got 4:02.50 in 400-meter individual medley"
print(f"Question: {textwrap.fill(QUESTION, 50)}")
history.add_user_message(QUESTION)

response = chat(history.messages)
print(f"Answer: {textwrap.fill(response.content, 50)}")


Question: Who holds the record in 400-meter individual
medley?
Answer: As of September 2021, the record in the 400-meter
individual medley for men is held by Michael
Phelps from the United States, with a time of
4:03.84 set on August 10, 2008, at the Beijing
Olympics. For women, the record is held by Ye
Shiwen from China, with a time of 4:28.43 set on
July 28, 2012, at the London Olympics.



Question: But I recently read Leon Marchand got 4:02.50 in
400-meter individual medley
Answer: I apologize for the incorrect information. As of
my last update in September 2021, the record I
provided was accurate. However, please note that
records can be broken or updated over time. If
Leon Marchand has indeed achieved a time of
4:02.50 in the 400-meter individual medley, it
would be a new record. I recommend checking with
reliable and up-to-date sources for the most
current information on swimming records.


In [3]:
# chains
from download_pdf import download_pdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
file_name = os.getcwd() + "/pdfs/" + os.getenv("SUMMARIZE_FILE_NAME")
download_pdf(os.getenv("SUMMARIZE_FILE_URL"), file_name)



loader = PyPDFLoader(file_name)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
print(textwrap.fill(chain.run(texts), 50))



PDF already downloaded


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Generative Agents: Interactive Simulacra of Human Behavior
Joon Sung Park
Stanford University
Stanford, USA
joonspk@stanford.eduJoseph C. O’Brien
Stanford University
Stanford, USA
jobrien3@stanford.eduCarrie J. Cai
Google Research
Mountain View, CA, USA
cjcai@google.com
Meredith Ringel Morris
Google Research
Seattle, WA, USA
merrie@google.comPercy Liang
Stanford University
Stanford, USA
pliang@cs.stanford.eduMichael S. Bernstein
Stanford University
Stanford, USA
msb@cs.stanford.edu
Figure 1: Generative agents create believable simulacra of human behavior for interactive applications. In this work, we demon-"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"strate generative agents by populating a sandbox environment, reminiscent of The Sims, with twenty-five agents. Users

' This paper introduces generative agents, a type of computational software, to create believable proxies of human behavior for interactive applications. It presents an agent architecture that uses a large language model to store experiences in natural language, synthesize memories into higher-level reflections, and retrieve them to plan behavior. It evaluates the use of natural language agents to generate believable individual and social behaviors, and discusses the use of parasocial relationships to mitigate risks from deepfakes and tailored persuasion. It references various sources and provides examples of how these techniques can be applied.'

In [18]:
# agents
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import Tool

def substract_tool(input):
    x, y = input.split("-")
    return str(int(x) - int(y))

toolkit = load_tools(["ddg-search"], llm=llm)
toolkit.append(Tool(
    name="substraction-tool",
    description="Substracts two numbers of the form 'x-y' and returns the result.",
    func=substract_tool
))

agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)
response = agent({"input":"what was the difference between the average temperature in vancouver canada summer vs winter"})



> Entering new AgentExecutor chain...
 I need to find out the average temperature in Vancouver for both summer and winter
Action: duckduckgo_search
Action Input: average temperature in vancouver canada summer vs winter
Observation: Summer, from June to August, is cool, but also quite sunny: the maximum temperatures hover between 20 °C and 23 °C (68 °F and 73 °F), but the Pacific high-pressure system provides longer periods of good weather than in the rest of Canada. If in winter, Vancouver is the mildest large city of Canada, in summer, it is the coolest, but also the ... The average annual temperature in Vancouver is 11.0 °C (51.8 °F) downtown and 10.4 °C (50.7 °F) at Vancouver International Airport in Richmond. This is one of the warmest in Canada. Greater Vancouver is in USDA plant hardiness zone 8, [2] similar to other coastal or near-coastal cities such as Seattle, Portland, Amsterdam, and London, as ... The weather in Vancouver is variable, with November typically being the rai